In [2]:
import pandas as pd

# Load each of the CSV files into its own DataFrame
df_airports = pd.read_csv('airports_geolocation.csv')
df_flights_jan2024 = pd.read_csv('maj us flight - january 2024.csv')
df_flights_2023 = pd.read_csv('US_flights_2023.csv', low_memory=False)
df_weather = pd.read_csv('weather_meteo_by_airport.csv')
df_cancelled_diverted = pd.read_csv('Cancelled_Diverted_2023.csv')

# --- Verification Step ---
# You can print the first few rows of any dataframe to make sure it loaded correctly
print("Successfully loaded 'US_flights_2023.csv'. Here's a preview:")
print(df_flights_2023.info())

Successfully loaded 'US_flights_2023.csv'. Here's a preview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6743404 entries, 0 to 6743403
Data columns (total 24 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   FlightDate          object
 1   Day_Of_Week         int64 
 2   Airline             object
 3   Tail_Number         object
 4   Dep_Airport         object
 5   Dep_CityName        object
 6   DepTime_label       object
 7   Dep_Delay           int64 
 8   Dep_Delay_Tag       int64 
 9   Dep_Delay_Type      object
 10  Arr_Airport         object
 11  Arr_CityName        object
 12  Arr_Delay           int64 
 13  Arr_Delay_Type      object
 14  Flight_Duration     int64 
 15  Distance_type       object
 16  Delay_Carrier       int64 
 17  Delay_Weather       int64 
 18  Delay_NAS           int64 
 19  Delay_Security      int64 
 20  Delay_LastAircraft  int64 
 21  Manufacturer        object
 22  Model               object
 23  Aicraft_age         i

In [7]:
import pandas as pd

# We assume the dataframes are already loaded from your previous step:
# df_airports, df_flights_jan2024, df_flights_2023, df_weather, df_cancelled_diverted

def inspect_df(name, df):
    print(f"\n=== INSPECTING: {name} ===")
    print(f"Shape: {df.shape}")
    print("Columns:", list(df.columns))
    print("First 3 Rows:")
    print(df.head(3))

inspect_df("2023 Flights", df_flights_2023)

# 1. Inspect Airports Geolocation (Looking for State, Lat/Lon for clustering)
inspect_df("Airports Geolocation", df_airports)

# 2. Inspect Weather Data (Looking for Wind, Temp, Rain to predict delays)
inspect_df("Weather Data", df_weather)

# 3. Inspect Cancelled/Diverted (Check if this has useful info or just IDs)
inspect_df("Cancelled/Diverted", df_cancelled_diverted)

# 4. Inspect Jan 2024 Data (Check if schema matches 2023 for testing)
inspect_df("Jan 2024 Flights", df_flights_jan2024)

# 5. Re-verify Main Flight Data 'Distance_type' just to be sure
print("\n=== VALUE COUNTS: Distance_type (Main File) ===")
print(df_flights_2023['Distance_type'].value_counts().head())


=== INSPECTING: 2023 Flights ===
Shape: (6743404, 24)
Columns: ['FlightDate', 'Day_Of_Week', 'Airline', 'Tail_Number', 'Dep_Airport', 'Dep_CityName', 'DepTime_label', 'Dep_Delay', 'Dep_Delay_Tag', 'Dep_Delay_Type', 'Arr_Airport', 'Arr_CityName', 'Arr_Delay', 'Arr_Delay_Type', 'Flight_Duration', 'Distance_type', 'Delay_Carrier', 'Delay_Weather', 'Delay_NAS', 'Delay_Security', 'Delay_LastAircraft', 'Manufacturer', 'Model', 'Aicraft_age']
First 3 Rows:
  FlightDate  Day_Of_Week       Airline Tail_Number Dep_Airport  Dep_CityName  \
0 2023-01-02            1  Endeavor Air      N605LR         BDL  Hartford, CT   
1 2023-01-03            2  Endeavor Air      N605LR         BDL  Hartford, CT   
2 2023-01-04            3  Endeavor Air      N331PQ         BDL  Hartford, CT   

  DepTime_label  Dep_Delay  Dep_Delay_Tag Dep_Delay_Type  ... Flight_Duration  \
0       Morning         -3              0      Low <5min  ...              56   
1       Morning         -5              0      Low <5min  

In [4]:
import pandas as pd

# --- Step 1: Prepare the main flights DataFrame ---
df_flights_2023['FlightDate'] = pd.to_datetime(df_flights_2023['FlightDate'])
df_weather['time'] = pd.to_datetime(df_weather['time'])

# --- Step 2: Create two distinct, renamed weather DataFrames ---

# Define the columns we want to rename
weather_cols = ['tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'pres', 'time', 'airport_id']

# Create a version for DEPARTURE weather
dep_rename_dict = {col: col + '_dep' for col in weather_cols}
df_weather_dep = df_weather.rename(columns=dep_rename_dict)

# Create a version for ARRIVAL weather
arr_rename_dict = {col: col + '_arr' for col in weather_cols}
df_weather_arr = df_weather.rename(columns=arr_rename_dict)


# --- Step 3: Perform the first merge with the departure weather data ---
df_merged = pd.merge(
    df_flights_2023,
    df_weather_dep,
    left_on=['FlightDate', 'Dep_Airport'],
    right_on=['time_dep', 'airport_id_dep'],
    how='left'
)

# --- Step 4: Perform the second merge with the arrival weather data ---
df_final = pd.merge(
    df_merged,
    df_weather_arr,
    left_on=['FlightDate', 'Arr_Airport'],
    right_on=['time_arr', 'airport_id_arr'],
    how='left'
)

# --- Step 5: Clean up the redundant key columns ---
df_final = df_final.drop(columns=['time_dep', 'airport_id_dep', 'time_arr', 'airport_id_arr'])


# --- Verification ---
# This will now show the correct '_dep' and '_arr' suffixes for all weather columns
print("Columns of the final merged DataFrame:")
print(df_final.info())

print("\nPreview of the final DataFrame with weather for both airports:")
print(df_final[['FlightDate', 'Dep_Airport', 'Arr_Airport', 'tavg_dep', 'tavg_arr']].head())

Columns of the final merged DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6743404 entries, 0 to 6743403
Data columns (total 40 columns):
 #   Column              Dtype         
---  ------              -----         
 0   FlightDate          datetime64[ns]
 1   Day_Of_Week         int64         
 2   Airline             object        
 3   Tail_Number         object        
 4   Dep_Airport         object        
 5   Dep_CityName        object        
 6   DepTime_label       object        
 7   Dep_Delay           int64         
 8   Dep_Delay_Tag       int64         
 9   Dep_Delay_Type      object        
 10  Arr_Airport         object        
 11  Arr_CityName        object        
 12  Arr_Delay           int64         
 13  Arr_Delay_Type      object        
 14  Flight_Duration     int64         
 15  Distance_type       object        
 16  Delay_Carrier       int64         
 17  Delay_Weather       int64         
 18  Delay_NAS           int64         
 19  Del

In [5]:
rename_mapping = {
    # Departure Weather Columns
    'tavg_dep': 'avg_temp_dep',
    'tmin_dep': 'min_temp_dep',
    'tmax_dep': 'max_temp_dep',
    'prcp_dep': 'precipitation_dep',
    'snow_dep': 'snowfall_dep',
    'wdir_dep': 'wind_direction_dep',
    'wspd_dep': 'wind_speed_dep',
    'pres_dep': 'pressure_dep',

    # Arrival Weather Columns
    'tavg_arr': 'avg_temp_arr',
    'tmin_arr': 'min_temp_arr',
    'tmax_arr': 'max_temp_arr',
    'prcp_arr': 'precipitation_arr',
    'snow_arr': 'snowfall_arr',
    'wdir_arr': 'wind_direction_arr',
    'wspd_arr': 'wind_speed_arr',
    'pres_arr': 'pressure_arr',
    
    # You could also rename other columns if you wish, for example:
    'Aicraft_age': 'Aircraft_Age'
}

# Apply the renaming to the DataFrame
df_final_renamed = df_final.rename(columns=rename_mapping)
print("Columns of the final merged DataFrame:")
print(df_final_renamed.info())

Columns of the final merged DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6743404 entries, 0 to 6743403
Data columns (total 40 columns):
 #   Column              Dtype         
---  ------              -----         
 0   FlightDate          datetime64[ns]
 1   Day_Of_Week         int64         
 2   Airline             object        
 3   Tail_Number         object        
 4   Dep_Airport         object        
 5   Dep_CityName        object        
 6   DepTime_label       object        
 7   Dep_Delay           int64         
 8   Dep_Delay_Tag       int64         
 9   Dep_Delay_Type      object        
 10  Arr_Airport         object        
 11  Arr_CityName        object        
 12  Arr_Delay           int64         
 13  Arr_Delay_Type      object        
 14  Flight_Duration     int64         
 15  Distance_type       object        
 16  Delay_Carrier       int64         
 17  Delay_Weather       int64         
 18  Delay_NAS           int64         
 19  Del

In [6]:
missing_values = df_final_renamed.isnull().sum()

# Filter to show only columns that have at least one missing value
missing_values = missing_values[missing_values > 0]

# Calculate the percentage of missing values for those columns
missing_percentage = (missing_values / len(df_final_renamed)) * 100

# Combine the count and percentage into a new DataFrame for a clean report
missing_report = pd.DataFrame({
    'Missing Count': missing_values,
    'Percentage (%)': missing_percentage
})

# Sort the report to show columns with the most missing values first
missing_report = missing_report.sort_values(by='Missing Count', ascending=False)

print("--- Missing Values Report ---")
if not missing_report.empty:
    print(missing_report)
else:
    print("Congratulations! No missing values found in the DataFrame.")
print("---------------------------\n")



--- Missing Values Report ---
Congratulations! No missing values found in the DataFrame.
---------------------------

